In [211]:
import pandas as pd, numpy as np
import sqlite3
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from unicodedata import normalize
from nltk.stem.snowball import SnowballStemmer
import re
from  nltk.corpus import stopwords

In [212]:
conn = sqlite3.connect("clientes4.db")
test = pd.read_sql("""select codigo as codigoCbo, printf('%s %s %s %s', nome ,descricao_sumaria, formacao_experiencia, condicoes_exercicio) as unificado
		from familia  
""", conn)

In [213]:
train = pd.read_csv("train.csv", sep = ";")
train

,codigoCbo,seguranca,relacionamento,saude,tecnologia,manufatura,gastronomia,politicos,artistas
0,101,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,211,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,203,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,302,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,212,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,303,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,311,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2410,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
train = train.merge(test, on='codigoCbo')

In [216]:
train

,codigoCbo,seguranca,relacionamento,saude,tecnologia,manufatura,gastronomia,politicos,artistas,unificado
0,101,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oficiais generais das forças armadas As funçõe...
1,211,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Subtenentes e sargentos da policia militar Com...
2,103,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Praças das forças armadas Estão compreendidas ...
3,201,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oficiais superiores da polícia militar Comanda...
4,203,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tenentes da polícia militar Comandam pelotão, ..."
5,302,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oficiais intermediários do corpo de bombeiros ...
6,212,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cabos e soldados da polícia militar Realizam p...
7,303,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tenentes do corpo de bombeiros militar Comanda...
8,311,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Subtenentes e sargentos do corpo de bombeiros ...
9,2410,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Advogados Postulam, em nome do cliente, em juí..."


In [217]:
test = test[~test['codigoCbo'].isin(train['codigoCbo'])]

In [218]:
test.head()

,codigoCbo,unificado
1,102,Oficiais das forças armadas Estão compreendida...
4,202,Capitães da polícia militar Comandam operaçõe...
8,301,Oficiais superiores do corpo de bombeiros mili...
13,1111,Legisladores Estudam as matérias previstas nas...
15,1113,Magistrados Decidem conflitos individuais e co...


In [219]:
COMMENT = 'unificado'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

train.head()

,codigoCbo,seguranca,relacionamento,saude,tecnologia,manufatura,gastronomia,politicos,artistas,unificado
0,101,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oficiais generais das forças armadas As funçõe...
1,211,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Subtenentes e sargentos da policia militar Com...
2,103,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Praças das forças armadas Estão compreendidas ...
3,201,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oficiais superiores da polícia militar Comanda...
4,203,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tenentes da polícia militar Comandam pelotão, ..."


In [221]:
def  clean_and_tokenizing(text):
    # Remove numbers + punctuation        
    letters_only = re.sub("[^a-zA-Z]", " ", normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')) 

    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             

    # Save english stopwords
    stops = set(stopwords.words("portuguese"))                  

    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # Join everython again
    #print(meaningful_words)
    print(meaningful_words)
    return( meaningful_words )

In [222]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=clean_and_tokenizing,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

['oficiais', 'generais', 'forcas', 'armadas', 'funcoes', 'deste', 'grupo', 'consistem', 'manter', 'cumprir', 'defender', 'constituicao', 'federal', 'dentro', 'esfera', 'atribuicoes', 'observar', 'leis', 'promover', 'condicoes', 'seguranca', 'necessarias', 'desenvolvimento', 'bem', 'estar', 'geral', 'defender', 'integridade', 'territorial', 'soberania', 'pais', 'dirigem', 'orientam', 'orgaos', 'superiores', 'forcas', 'armadas', 'planejam', 'conduzem', 'diversos', 'assun', 'tos', 'area', 'militar', 'assessoram', 'realizacao', 'politicas', 'publicas', 'esfera', 'federal', 'geralmente', 'oficiais', 'generais', 'contam', 'anos', 'experiencia', 'profissional', 'possuem', 'cursos', 'altos', 'estudos', 'militares', 'politica', 'estrategia', 'feitos', 'escola', 'superior', 'guerra', 'esg', 'escolas', 'altos', 'estudos', 'militares', 'cada', 'forca', 'armada', 'quais', 'equivalem', 'doutorado', 'area', 'civil', 'trabalho', 'oficial', 'general', 'exercido', 'organizacoes', 'militares', 'instituic

['psicologos', 'psicanalistas', 'estudam', 'pesquisam', 'avaliam', 'desenvolvimento', 'emocional', 'processos', 'mentais', 'sociais', 'individuos', 'grupos', 'instituicoes', 'finalidade', 'analise', 'tratamento', 'orientacao', 'educacao', 'diagnosticam', 'avaliam', 'disturbios', 'emocionais', 'mentais', 'adaptacao', 'social', 'elucidando', 'conflitos', 'questoes', 'acompanhando', 's', 'paciente', 's', 'durante', 'processo', 'tratamento', 'cura', 'investigam', 'fatores', 'inconscientes', 'comportamento', 'individual', 'grupal', 'tornando', 'conscientes', 'desenvolvem', 'pesquisas', 'experimentais', 'teoricas', 'clinicas', 'coordenam', 'equipes', 'atividades', 'area', 'afins', 'trabalhadores', 'dessa', 'familia', 'exigido', 'nivel', 'superior', 'completo', 'experiencia', 'profissional', 'varia', 'segundo', 'formacao', 'psicologos', 'modo', 'geral', 'pede', 'quatro', 'anos', 'caso', 'psicologo', 'clinico', 'psicanalista', 'necessario', 'minimo', 'cinco', 'anos', 'experiencia', 'cursos', '

['trabalhadores', 'atencao', 'defesa', 'protecao', 'pessoas', 'situacao', 'risco', 'adolescentes', 'conflito', 'lei', 'visam', 'garantir', 'atencao', 'defesa', 'protecao', 'pessoas', 'situacoes', 'risco', 'pessoal', 'social', 'adolescentes', 'conflito', 'lei', 'procuram', 'assegurar', 'direitos', 'abordando', 'sensibilizando', 'identificando', 'necessidades', 'demandas', 'controlam', 'acesso', 'pessoas', 'veiculos', 'unidade', 'penal', 'conduzem', 'presos', 'internados', 'desenvolvimento', 'atividades', 'culturais', 'esportivas', 'escolares', 'laborativas', 'recreativas', 'ressocializadoras', 'acesso', 'ocupacoes', 'familia', 'livre', 'requisitos', 'escolaridade', 'caso', 'monitor', 'ressocializacao', 'prisional', 'exige', 'segundo', 'grau', 'completo', 'curso', 'basico', 'caso', 'socioeducadores', 'exige', 'segundo', 'grau', 'completo', 'ocupacao', 'conselheiro', 'tutelar', 'observa', 'diversidade', 'bastante', 'acentuada', 'diz', 'respeito', 'escolaridade', 'pode', 'variar', 'ensino'

['trabalhadores', 'arte', 'acabamento', 'madeira', 'mobiliario', 'elaboram', 'projetos', 'acabamento', 'madeira', 'mobiliario', 'planejam', 'trabalho', 'organizam', 'local', 'execucao', 'preparam', 'maquinas', 'ferramentas', 'preparam', 'superficies', 'realizam', 'acabamento', 'madeiras', 'moveis', 'conforme', 'normas', 'procedimentos', 'tecnicos', 'qualidade', 'seguranca', 'meio', 'ambiente', 'saude', 'podem', 'aplicar', 'inumeros', 'processos', 'tecnicas', 'acabamento', 'dentre', 'marchetaria', 'entalhe', 'lustracao', 'folheacao', 'laqueacao', 'decape', 'marmorizacao', 'exercicio', 'ocupacoes', 'requer', 'normalmente', 'ensino', 'fundamental', 'exercicio', 'pleno', 'atividades', 'ocorre', 'apos', 'tres', 'quatro', 'anos', 'pratica', 'podem', 'aprender', 'trabalho', 'acompanhando', 'profissional', 'experiente', 'fazendo', 'treinamentos', 'ministrados', 'instituicoes', 'formacao', 'profissional', 'empresas', 's', 'ocupacao', 'oes', 'elencada', 's', 'nesta', 'familia', 'ocupacional', 'd

In [223]:
trn_term_doc
test_term_doc

<573x719 sparse matrix of type '<class 'numpy.float64'>'
	with 66584 stored elements in Compressed Sparse Row format>

In [224]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [225]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [226]:
x = trn_term_doc

test_x = test_term_doc

In [227]:
train.fillna(0, inplace= True)

In [228]:
label_cols = list(train)[1:9]

In [229]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit seguranca
fit relacionamento
fit saude
fit tecnologia
fit manufatura
fit gastronomia
fit politicos
fit artistas


In [230]:
submid = pd.DataFrame({'id': test["codigoCbo"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

In [46]:
list(trn_term_doc)

[<1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 71 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 108 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 74 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 66 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 67 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 106 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 100 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 147 stored elements in Compressed Sparse Row format>,
 <1x537 sparse matrix of type '<class 'numpy.float64'>'
 	with 142 s